### Unified Inference and Visualization for YOLO Model Variants
This notebook demonstrates running different flavors of YOLO models (e.g., image classification, object detection, pose estimation, and segmentation) 
using a unified codebase with the DeGirum PySDK and tools. Key features include:

- Unified handling of YOLO model variants with minimal changes to the code.
- Flexible selection of inference host (cloud or local) and model zoo location.
- Seamless output visualization, regardless of the specific YOLO model used.

Simply uncomment a model of your choice, provide the necessary configurations, and run the code block 
to perform inference and visualize the results.

In [1]:
import degirum as dg, degirum_tools

# choose a model to run inference on by uncommenting one of the following lines
model_name = "yolov8n_relu6_coco--640x640_quant_hailort_hailo8l_1"
# model_name = "yolov8n_relu6_coco_pose--640x640_quant_hailort_hailo8l_1"
# model_name = "yolov8n_relu6_coco_seg--640x640_quant_hailort_hailo8l_1"
# model_name = "yolov8s_silu_imagenet--224x224_quant_hailort_hailo8l_1"

# choose inference host address
inference_host_address = "@cloud" 
# inference_host_address = "@local"

# choose zoo_url
zoo_url = "degirum/models_hailort"
# zoo_url = "<path to local folder>"

# choose image source
image_source = "../assets/ThreePersons.jpg"

# set token
token = degirum_tools.get_token()
# token = '' # leave empty for local inference

# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token
)

# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image source '{image_source}'")
inference_result = model(image_source)

# print('Inference Results \n', inference_result)  # numeric results
print(inference_result)
print("Press 'x' or 'q' to stop.")

# show results of inference
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(inference_result)


 Running inference using 'yolov8n_relu6_coco--640x640_quant_hailort_hailo8l_1' on image source '../assets/ThreePersons.jpg'
- bbox: [50.34862518310547, 11.273429870605469, 259.01898193359375, 421.975830078125]
  category_id: 0
  label: person
  score: 0.9210436940193176
- bbox: [425.5379333496094, 20.12468719482422, 640.0, 353.85784912109375]
  category_id: 0
  label: person
  score: 0.888812780380249
- bbox: [217.6569366455078, 45.100433349609375, 453.69573974609375, 402.4808654785156]
  category_id: 0
  label: person
  score: 0.8193221092224121

Press 'x' or 'q' to stop.


In [25]:
from sklearn.cluster import KMeans
import numpy as np
def rearrange_detections(detections):
    
    # Extract y-centers of bounding boxes
    y_centers = np.array([[(det["bbox"][1] + det["bbox"][3]) / 2] for det in detections])

    # Check if all y-centers are within a small range (indicating a single line)
    if len(detections) <= 1 or np.max(y_centers) - np.min(y_centers) < 20:  # Adjust threshold as needed
        # Sort detections by the x_min value
        detections_sorted = sorted(detections, key=lambda det: det["bbox"][0])
        # Concatenate all labels into a single line
        first_line_labels = "".join([det["label"] for det in detections_sorted])
        second_line_labels = ""  # No second line
    else:
        # Apply KMeans clustering to group detections into two lines
        kmeans = KMeans(n_clusters=2, random_state=0).fit(y_centers)
        labels = kmeans.labels_

        # Separate detections into two lines based on clustering labels
        line_0 = [det for i, det in enumerate(detections) if labels[i] == 0]
        line_1 = [det for i, det in enumerate(detections) if labels[i] == 1]

        # Determine the order of lines based on the average y-center
        avg_y_line_0 = np.mean([((det["bbox"][1] + det["bbox"][3]) / 2) for det in line_0])
        avg_y_line_1 = np.mean([((det["bbox"][1] + det["bbox"][3]) / 2) for det in line_1])

        if avg_y_line_0 < avg_y_line_1:
            first_line = line_0
            second_line = line_1
        else:
            first_line = line_1
            second_line = line_0

        # Sort each line by the x_min value (first value in bbox)
        first_line_sorted = sorted(first_line, key=lambda det: det["bbox"][0])
        second_line_sorted = sorted(second_line, key=lambda det: det["bbox"][0])

        # Concatenate all labels in sorted order for each line
        first_line_labels = "".join([det["label"] for det in first_line_sorted])
        second_line_labels = "".join([det["label"] for det in second_line_sorted])

    # Return the labels for both lines
    return first_line_labels, second_line_labels


In [26]:
import degirum as dg, degirum_tools

# choose a model to run inference on by uncommenting one of the following lines
model_name = "yolov8n_relu6_lp_ocr--256x128_float_openvino_multidevice_1"
# model_name = "yolov8n_relu6_coco_pose--640x640_quant_hailort_hailo8l_1"
# model_name = "yolov8n_relu6_coco_seg--640x640_quant_hailort_hailo8l_1"
# model_name = "yolov8s_silu_imagenet--224x224_quant_hailort_hailo8l_1"

# choose inference host address
inference_host_address = "@cloud" 
# inference_host_address = "@local"

# choose zoo_url
zoo_url = "degirum/models_openvino"
# zoo_url = "<path to local folder>"

# choose image source
image_source = "C://Users//ShashiChilappagari//Downloads//rh2.jpeg"

# set token
token = degirum_tools.get_token()
# token = '' # leave empty for local inference

# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    output_confidence_threshold=0.2
)

# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image source '{image_source}'")
inference_result = model(image_source)

# print('Inference Results \n', inference_result)  # numeric results
# print(inference_result)
# print("Press 'x' or 'q' to stop.")

labels1, labels2= rearrange_detections(inference_result.results)
print(labels1, labels2)
# # show results of inference
# with degirum_tools.Display("AI Camera") as output_display:
#     output_display.show_image(inference_result)


 Running inference using 'yolov8n_relu6_lp_ocr--256x128_float_openvino_multidevice_1' on image source 'C://Users//ShashiChilappagari//Downloads//rh2.jpeg'
123456 


In [17]:
inference_result

[]

In [27]:
import os
import cv2
import json

def rearrange_detections(detections):
    
    # Sort the detections by the x_min value (first value in bbox)
    sorted_detections = sorted(detections, key=lambda det: det["bbox"][0])

    # Concatenate all labels in sorted order
    concatenated_labels = "".join([det["label"] for det in sorted_detections])

    # Return the concatenated labels
    return concatenated_labels

def run_inference_on_folder(input_folder, output_folder, model):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_file in os.listdir(input_folder):
        if image_file.lower().endswith((".jpg", ".jpeg", ".png")):
            # Load image
            image_path = os.path.join(input_folder, image_file)
            print(image_path)
            # image = cv2.imread(image_path)

            # Run inference (replace this with the actual inference function)
            results = model(image_path)

            # Rearrange detections and get concatenated labels
            concatenated_labels = rearrange_detections(results.results)

            # Overlay labels on the image
            annotated_image = results.image.copy()
            height, width, _ = annotated_image.shape

            band_height = 50
            annotated_image = cv2.copyMakeBorder(results.image, 0, band_height, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

            # Add text on the black band
            cv2.putText(
                annotated_image,
                concatenated_labels,
                (10, height + band_height - 15),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2
            )

            # Save the annotated image
            output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + "_annotated.jpg")
            cv2.imwrite(output_path, annotated_image)

In [30]:
# Example usage
input_folder = "C:\\Users\\ShashiChilappagari\\Documents\\Python_Scripts\\Datasets\\FiftyStatesLicPlates\\FiftyStatesLicensePlates"
output_folder = "C:\\Users\\ShashiChilappagari\\Documents\\Python_Scripts\\Datasets\\FiftyStatesLicPlates\\annotated"
model.output_confidence_threshold = 0.2
run_inference_on_folder(input_folder, output_folder, model)

C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Alabama.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Alaska.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Arizona.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Arkansas.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\California.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Colorado.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Connecticut.jpg
C:\Users\ShashiChilappagari\Documents\Python_Scripts\Datasets\FiftyStatesLicPlates\FiftyStatesLicensePlates\Delaware.jpg
C:\Users\ShashiChilappagari\Doc